### 협업일지
#### 일자 : 2024-11-26 / 박하진

#### 오늘 할 일
-
-
-

#### 오늘 한 일
- 선별된 자치구 중 중구를 맡아 데이터 EDA 진행
- 시간대 별 거치대수량 top 10을 뽑아 시각화 한 후 상세 조사할 대여소 명 선별
-

#### 내일 할 일
-
-
-

#### Codes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
import os
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("선별자치구_자치구별_대여가능자전거_거치대수.csv", encoding='CP949')
df.head(3)

In [ ]:
# 'Unnamed: 0' 컬럼 삭제
df = df.drop(columns='Unnamed: 0')

In [ ]:
region_df = df[df['자치구'] == '중구']
region_df.head(3)

region_df.to_csv('region_df.csv', index=False)

In [ ]:
tmp = region_df.groupby(['보관소(대여소)명','시간대']).mean(numeric_only=True)[['거치대수량','거치대수']]
tmp.head(24)

In [ ]:
tmp = region_df.groupby(['보관소(대여소)명']).mean(numeric_only=True)[['거치대수량','거치대수']]
tmp['수요지수'] = (tmp['거치대수'] - tmp['거치대수량'])/tmp['거치대수']
tmp.sort_values(by='수요지수',ascending=False).head(10)

In [ ]:
places = ['484.도동삼거리', '469.회현역 1번출구', '470.회현역 7번출구', '4772. 중앙일보', '4776. 신세계면세점', '322. 명동성당 앞', '435. SK 남산빌딩', '4753. 시청역 10번출구', '321. KEB 하나금융그룹 명동사옥 옆','4752. 삼성사이버빌리지아파트']

tmp_10 = region_df[region_df['보관소(대여소)명'].isin(places)].copy()

In [ ]:
places = ['484.도동삼거리', '469.회현역 1번출구', '470.회현역 7번출구', '4772. 중앙일보', '4776. 신세계면세점', '322. 명동성당 앞', '435. SK 남산빌딩', '4753. 시청역 10번출구', '321. KEB 하나금융그룹 명동사옥 옆','4752. 삼성사이버빌리지아파트']

plt.rc('font', family='Malgun Gothic')

for place in places:
    tmp_df = tmp_10[tmp_10['보관소(대여소)명'] == place]
    tmp_df["변화량"] = tmp_df["거치대수량"].diff().fillna(0).abs()

    #하이라이트 기준 점
    threshold = 0.4
    highlight = tmp_df[tmp_df["변화량"] > threshold]

    #평균값 계산
    mean_value = tmp_df["거치대수량"].mean()

    plt.figure(figsize=(10,4))
    sns.lineplot(tmp_df,x='시간대',y='거치대수량')

    # 평균선 추가
    plt.axhline(y=mean_value, color="green", linestyle="--", label=f"평균선: {mean_value:.2f}")
    plt.xticks(range(0, 24))
    plt.title(f'시간대별 대여가능 자전거 수 추이 - {place}')
    plt.ylim(0,10)
    plt.grid(axis="x", linestyle="--", alpha=0.6)
    plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

selected_places = ['484.도동삼거리', '469.회현역 1번출구', '470.회현역 7번출구', '4752. 삼성사이버빌리지아파트']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))  
axes = axes.flatten()  

for i, place in enumerate(selected_places):
    tmp_df = tmp_10[tmp_10['보관소(대여소)명'] == place].copy()
    tmp_df["변화량"] = tmp_df["거치대수량"].diff().fillna(0).abs()
    mean_value = tmp_df["거치대수량"].mean()

    ax = axes[i] 
    sns.lineplot(data=tmp_df, x='시간대', y='거치대수량', ax=ax, label="거치대수량 변화 추이")
    ax.axhline(y=mean_value, color="green", linestyle="--", label=f"평균선: {mean_value:.2f}")

    ax.set_title(f'{place}', fontsize=12)
    ax.set_xticks(range(0, 24))  
    ax.set_ylim(0, 10) 
    ax.grid(axis="x", linestyle="--", alpha=0.6)
    ax.legend()

plt.tight_layout() 
plt.show()

# 평균 거치대 수량이 2인 회현역 1번, 7번 출구 대여소를 개선 대상으로 지정 
### (도동삼거리는 수요지수가 1위지만 현재 폐쇄된 대여소, 다음 순위인 회현역1번,7번출구를 개선 대상 대여소로 선정)

## 1. 회현역 주변 특성
- 회현역은 서울의 중심에 위치하며, 상업 및 업무지구가 밀집된 지역으로, 대중교통 접근성이 뛰어납니다.
    - 주변에 남대문시장, 롯데백화점 본점, 명동 쇼핑거리 등이 위치하여 관광객 및 쇼핑객이 많습니다.
    - 퇴근 시간대에는 직장인들의 이동과 더불어 주변 상업시설 방문 수요가 겹쳐 자전거 대여 수요가 집중될 가능성이 있습니다.
- 회현역이 서울역과 인접하여, 기차/공항철도를 이용하는 사람들의 마지막 이동 수단으로 따릉이가 사용될 가능성이 높습니다.
    - 특히, 공항철도를 이용하는 관광객들이 서울역으로 가기위해 이용 시 따릉이를 사용하는 경우, 회현역에서 자전거 반납이 이루어지지 않고 다른 대여소로 이동할 수 있습니다.

## 2. 출, 퇴근 시간대 수요 감소와 대여소 공급 부족
- 중구의 높은 주간인구지수는 직장인 및 상업 활동 인구가 많다는 것을 의미하며, 퇴근 시간대에 인구가 급격히 빠져나간다는 점을 나타냅니다.
    - 이로 인해 회현역 1번, 7번 출구 대여소에서 퇴근 후 대여 수요가 폭발적으로 증가하면서 거치대수량이 급격히 감소합니다.
    - 회현역 1번출구와 7번출구의 출퇴근 시간대(7-9시, 18-20시)를 보면 출근시간 때에는 평균선보다 아래에 존재하고, 퇴근시간때는 음의 기울기로 감소하는 모습을 볼 수 있습니다.
- 거치대 부족 문제:
    - 대여소의 총 거치대 수가 수요를 감당하지 못해 자전거 반납이 불가능한 경우, 다른 대여소로 이동하게 되어 반납 분산 현상이 발생할 수 있습니다.
    - 상위권 10개 구역의 평균 거치대 수는 18.6개로 회현역 1번과 7번 출구 대여소의 각각의 거치대 수는 11개와 13개가 있습니다.

## 3. 인근 대여소와의 관계
- 회현역 1번, 7번 출구 대여소는 모두 서울역과 가까운 위치에 있어, 주변 대여소 간 자전거 흐름 불균형이 발생할 가능성이 큽니다.
    - 서울역 주변 대여소에서 자전거가 소진되고, 반납은 외곽 대여소에 집중되는 현상이 나타날 수 있습니다.

#### Issues & Challenges
-
-
-

#### Reflection
-
-
-